# The `Bayes Theorem` 

In [13]:
#import the librairies pandas and numpy
import pandas as pd
import numpy as np

Do you remember this theorem covered during the lecture ? 

* The Bayes Theorem allows you to compute `a conditional probability`.
* It is widely used in Machine Learning to `update your knowledge`
* Despite its pretty simple formula, it can `highlight unexpected insights`

🧑🏻‍🏫 What is the `Bayes Theorem` ? According to [Brilliant](https://brilliant.org/wiki/bayes-theorem/) :

> Bayes' theorem is a formula that describes how to update the probabilities of hypotheses (A) when given evidence (Data).


🧮 The formula is the following:

$$ \mathbb{P}(A | Data) =  \mathbb{P}(A) \times \frac{\mathbb{P}(Data | A) }{\mathbb{P}(Data)}$$

## 0) Challenge context: Should we play sport outside expecting some weather conditions ?

* In this challenge, we'll try to recompute this formula.

* We have a dataset with `weather conditions` (Rain, Sunny, Overcast) and `play` (Yes, No) suggesting whether a sport game should be played based on the weather conditions.

In [14]:
weather_data_example = ['Sunny', 'Overcast', 'Rainy', 'Sunny', 'Sunny', 'Overcast', 'Rainy', 'Rainy', 'Sunny',
'Rainy', 'Sunny', 'Overcast', 'Overcast', 'Rainy']

play_data_example = ['No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No', 'No', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'No']

#create a dict data where the first (key, value) is weather and weather_data_example
#and second (key, value) is play and play_data_example

data = {"weather" : weather_data_example,
       "play" : play_data_example}

# convert data into a data frame df and print df

data_df = pd.DataFrame(data)
print(data_df)



     weather play
0      Sunny   No
1   Overcast  Yes
2      Rainy  Yes
3      Sunny  Yes
4      Sunny  Yes
5   Overcast  Yes
6      Rainy   No
7      Rainy   No
8      Sunny  Yes
9      Rainy  Yes
10     Sunny   No
11  Overcast  Yes
12  Overcast  Yes
13     Rainy   No


🚀 Let's compute $ \large P(play \mid weather) = P(play) \times \frac{P(weather \mid play)}{P(weather)} $

## 1) Warm-up : understanding the numbers with a `frequency table`

✍️ Grab a pen + a piece of paper and complete the **`frequency table`**:

| Weather  | Played | No | Total |
|----------|------|----|-------|
| Sunny    |     |   |      |
| Overcast |     |   |      |
| Rainy    |     |   |      |
| Total    |     |   |   14  |

<details>
    <summary>Answer here</summary>
    
| Weather  | Played | No | Total |
|----------|------|----|-------|
| Sunny    | 3    | 2  | 5     |
| Overcast | 4    | 0  | 4     |
| Rainy    | 2    | 3  | 5     |
| Total    | 9    | 5  | 14    |     
</details>

## 2) Prior probability : $ \mathbb{P}(play)$

🤔 What is the theoretical probability of a game being played ❓

Look at the numbers in your previous table.

<details>
    <summary>Answer</summary>
    
| Weather  | Played | No | Total |
|----------|------|----|-------|
| Total    | 9    | 5  | 14    |     
    
$ \mathbb{P}(played) = \frac{9}{14} = 64.29 \% $
</details>

👩🏻‍💻 Code the `prior_probability` function to compute the result.

In [15]:
def prior_probability(played: str, play_data: list) -> float:
    '''
    Returns P(played)
    Here we code this function using pandas
    '''
    filtered_df = data_df[data_df["play"] == played]
    count = len(filtered_df.index)
    return count/len(play_data)
    
    
# 👀 Run the following to test your code.
# If nothing shows, your function works. Otherwise, inspect your code to fix it!
assert(round(prior_probability("Yes", play_data_example),4) == 0.6429)

In [16]:
# A PYTHONIC SOLUTION  ( list comprehension)
def prior_probability_pythonic(played: str, play_data: list) -> float:
    """Same thing with a list comprehension"""
    count = 0
    for i in play_data:
        if i == played: count+=1
    
    return count/len(play_data)

# 👀 Run the following to test your code.
# If nothing shows, your function works. Otherwise, inspect your code to fix it!
assert(round(prior_probability_pythonic("Yes", play_data_example),4) == 0.6429)

In [17]:
# AN EVEN SHORTER SOLUTION WITH NUMPY
def prior_probability_numpy_ic(played: str, play_data: list) -> float:
    """Calculating probability with numpy"""
    data_array = np.array(play_data)
    values, counts = np.unique(data_array, return_counts=True)
    occurences = 0
    for v, c in zip(values, counts): 
        if v == played: 
            occurences = c
            break
    
    return occurences/len(play_data)

# 👀 Run the following to test your code.
# If nothing shows, your function works. Otherwise, inspect your code to fix it!
assert(round(prior_probability_numpy_ic("Yes", play_data_example),4) == 0.6429)

☝️ FYI: These strange notations
```python
def prior_probability(played: bool, play_data: list) -> float:
```
are called **typing hints**

They are optional in python, and used to let the reader know what type of arguments and output the function should accept/return. 

There also exist python libraries that enforce respect for these types, and raise error when not. 
It's a good practice to use these hits in very large projects to make sure nothing breaks silently.



## 3) Likelihood :  $ \mathbb{P}(weather | play)$

🤔 What is the theoretical probability of the weather being rainy knowing that a game was NOT played ❓

Look at the numbers in your previous table.

<details>
    <summary>Answer</summary>
    
| Weather  | No | 
|----------|----|
| Sunny    | 2  | 
| Overcast | 0  | 
| Rainy    | 3  | 
| Total    | 5  |         
    
$ \mathbb{P}(play) = \frac{3}{5} = 60 \% $
</details>

In [37]:
def likelihood(weather, played):
    """"TO DO: return P(weather|play)"""
    
    # Here we reuse the data_df dataframe created earlier and filter it
    # to keep only the lines where the 'play' value is equel to played
    filtered_df = data_df[data_df["play"] == played]
    # Now we count how many times weather appeared in the weather 
    # column of the filtered dataframe
    try:
        weather_count = filtered_df['weather'].value_counts()[weather]
    # This script will raise KeyError if weather isn't present in the
    # filtered dataframe, so we need to handle that
    except KeyError:
        weather_count = 0
    # Now we rcompute and return the probability
    return weather_count/len(filtered_df)
    
# 👀 Run the following to test your code.
# If nothing shows, your function works. Otherwise, inspect your code to fix it!
assert(likelihood("Rainy", "No") == 0.60)

## 4) Posterior probability : $ \large P(play \mid weather ) = P(play) \times \frac{P(weather \mid play)}{P(weather)} $

🔥 We can finally compute the posterior probability as: 

$$\large \text{posterior probability} = \text{prior probability} \times \text{likelihood} \times \beta $$ 

where $ \large \beta = \frac{1}{P(weather)} $ is _normalization factor_.
 
<details>
    <summary>Expected results</summary>

Remember the table that you completed earlier ? 

| Weather  | Played | No | Total |
|----------|------|----|-------|
| Sunny    | 3    | 2  | 5     |
| Overcast | 4    | 0  | 4     |
| Rainy    | 2    | 3  | 5     |
| Total    | 9    | 5  | 14    |   
    
Based on this table, we can compute $ \mathbb{P}(played | weather) $
    
| Weather  | Proba(Played\|Weather) | Proba(No\|Weather) |
|----------|----------------------|--------------------|
| Sunny    | 0.6                  | 0.4                |
| Overcast | 1.0                  | 0.0                |
| Rainy    | 0.4                  | 0.6                |
    
</details>

In [40]:
def posterior_probability(played, weather):
    '''TO DO: return P(play|weather)
    '''
    p_play = prior_probability_pythonic(played, play_data_example)
    p_weather = prior_probability_pythonic(weather, weather_data_example)
    p_weather_play = likelihood(weather, played)
    
    return p_play * p_weather_play / p_weather

In [42]:
# 👀 Run the following cell to test your code
assert(posterior_probability("Yes", "Sunny")==0.6)
assert(posterior_probability("No", "Sunny")==0.4)
assert(posterior_probability("Yes", "Overcast")==1.0)
assert(posterior_probability("No", "Overcast")==0)
assert(round(posterior_probability("Yes", "Rainy"),1)==0.4)
assert(posterior_probability("No", "Rainy")==0.6)

## 5) Taking a step back to understand what you've done

Thanks to what you’ve learned in this challenge, could you answer these questions :

1. _"Matches are more likely to be played than not if the weather is sunny"_ 👉 Is this statement correct ?
2. If you know for sure that it will be raining during the next game 🤔, what is your best guess (probability) that the game will be canceled ?

In [43]:
"""Are matches more likely to be played than not if the weather is sunny?
This script answers that question."""

p_play_if_sunny = posterior_probability("Yes", "Sunny")
print(f"When the weather is sunny, matches have a {p_play_if_sunny * 100}% chance to be played")
if p_play_if_sunny > 0.5:
    print("So matches are more likely to be played if the weather is sunny.")
elif p_play_if_sunny == 0.5:
    print("So matches have a 50/50 chance of being played if the weather is sunny.")
else:
    print("So matches are rather unlikely to be played if the weather is sunny.")


When the weather is sunny, matches have a 60.0% chance to be played
So matches are more likely to be played if the weather is sunny


In [44]:
"""This code finds out the probability a game will be canceled, provided that it rains"""

p_canceled_if_raining = posterior_probability("No", "Rainy")
print(f"The game has a {p_canceled_if_raining * 100}% chance to be canceled.")


The game has a 60.0% chance to be canceled


🏁 Congrats, you have a better idea of how the `Bayes formula` work !

💾 Do not forget to `add/commit/push` your notebook



▶️ If you are curious and/or impatient, you can already watch the [15-min Youtube video from 3Blue1Brown](https://www.youtube.com/watch?v=HZGCoVF3YvM) that we already mentioned in the lecture.